# Linear regression

In [1]:
using StatisticalRethinking
using DynamicHMC, TransformVariables, LogDensityProblems, MCMCDiagnostics
using Parameters, ForwardDiff

ProjDir = rel_path("..", "scripts", "05")
cd(ProjDir)

Import the dataset.

### snippet 5.4

In [2]:
wd = CSV.read(rel_path("..", "data", "WaffleDivorce.csv"), delim=';')
df = convert(DataFrame, wd);

mean_ma = mean(df[:Marriage])
df[:Marriage_s] = convert(Vector{Float64},
  (df[:Marriage]) .- mean_ma)/std(df[:Marriage]);

mean_mam = mean(df[:MedianAgeMarriage])
df[:MedianAgeMarriage_s] = convert(Vector{Float64},
  (df[:MedianAgeMarriage]) .- mean_mam)/std(df[:MedianAgeMarriage]);
# Show the first six rows of the dataset.
first(df[[1, 7, 14,15]], 6)

,Location,Divorce,Marriage_s,MedianAgeMarriage_s
,String⍰,Float64⍰,Float64,Float64
1,Alabama,12.7,0.0226441,-0.60629
2,Alaska,12.5,1.5498,-0.686699
3,Arizona,10.8,0.0489744,-0.204241
4,Arkansas,13.5,1.65512,-1.41039
5,California,8.0,-0.266989,0.599857
6,Colorado,11.6,0.891544,-0.284651


Model ``y ∼ Xβ + ϵ``, where ``ϵ ∼ N(0, σ²)`` IID. Student prior on σ

In [3]:
struct m_5_3{TY <: AbstractVector, TX <: AbstractMatrix}
    "Observations."
    y::TY
    "Covariates"
    X::TX
end

Make the type callable with the parameters *as a single argument*.

In [4]:
function (problem::m_5_3)(θ)
    @unpack y, X, = problem   # extract the data
    @unpack β, σ = θ            # works on the named tuple too
    ll = 0.0
    ll += logpdf(Normal(10, 10), X[1]) # a = X[1]
    ll += logpdf(Normal(0, 1), X[2]) # b1 = X[2]
    ll += logpdf(Normal(0, 1), X[3]) # b1 = X[3]
    ll += logpdf(TDist(1.0), σ)
    ll += loglikelihood(Normal(0, σ), y .- X*β)
    ll
end

Instantiate the model with data and inits.

In [5]:
N = size(df, 1)
X = hcat(ones(N), df[:Marriage_s], df[:MedianAgeMarriage_s]);
y = convert(Vector{Float64}, df[:Divorce])
p = m_5_3(y, X);
p((β = [1.0, 2.0, 3.0], σ = 1.0))

-2222.175273500088

Write a function to return properly dimensioned transformation.

In [6]:
problem_transformation(p::m_5_3) =
    as((β = as(Array, size(p.X, 2)), σ = asℝ₊))
# Wrap the problem with a transformation, then use Flux for the gradient.
P = TransformedLogDensity(problem_transformation(p), p)
∇P = ADgradient(:ForwardDiff, P);

Tune and sample.

In [7]:
chain, NUTS_tuned = NUTS_init_tune_mcmc(∇P, 1000);

MCMC, adapting ϵ (75 steps)
0.0011 s/step ...done
MCMC, adapting ϵ (25 steps)
0.0001 s/step ...done
MCMC, adapting ϵ (50 steps)
0.0019 s/step ...done
MCMC, adapting ϵ (100 steps)
5.3e-5 s/step ...done
MCMC, adapting ϵ (200 steps)
0.00072 s/step ...done
MCMC, adapting ϵ (400 steps)
3.6e-5 s/step ...done
MCMC, adapting ϵ (50 steps)
0.00021 s/step ...done
MCMC (1000 steps)
5.5e-5 s/step ...done


We use the transformation to obtain the posterior from the chain.

In [8]:
posterior = TransformVariables.transform.(Ref(∇P.transformation), get_position.(chain));
posterior[1:5]

5-element Array{NamedTuple{(:β, :σ),Tuple{Array{Float64,1},Float64}},1}:
 (β = [9.65689, 0.569203, -0.414063], σ = 1.7511511486886908) 
 (β = [9.36907, 0.123375, -1.17755], σ = 1.5991584713059104)  
 (β = [10.0059, -0.168695, -0.951555], σ = 1.3757908714866351)
 (β = [9.38269, 0.0685835, -0.88941], σ = 1.740348249504998)  
 (β = [9.93383, -0.158062, -1.20754], σ = 1.5041680958362125) 

Extract the parameter posterior means: `β`,

In [9]:
posterior_β = mean(first, posterior)

3-element Array{Float64,1}:
  9.682783032767357  
 -0.20023702766644882
 -1.232318182362815  

then `σ`:

In [10]:
posterior_σ = mean(last, posterior)

1.501953541216366

Effective sample sizes (of untransformed draws)

In [11]:
ess = mapslices(effective_sample_size,
                get_position_matrix(chain); dims = 1)
# NUTS-specific statistics
NUTS_statistics(chain)

Hamiltonian Monte Carlo sample of length 1000
  acceptance rate mean: 0.94, min/25%/median/75%/max: 0.55 0.91 0.97 0.99 1.0
  termination: AdjacentTurn => 13% DoubledTurn => 87%
  depth: 1 => 1% 2 => 32% 3 => 67%


cmdstan result

In [12]:
cmdstan_result = "
Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
          Mean        SD       Naive SE       MCSE      ESS
    a  9.69137275 0.21507432 0.0034006235 0.0038501180 1000
   bA -1.12184710 0.29039965 0.0045916216 0.0053055477 1000
   bM -0.12106472 0.28705400 0.0045387223 0.0051444688 1000
sigma  1.52326545 0.16272599 0.0025729239 0.0034436330 1000

Quantiles:
         2.5%       25.0%      50.0%      75.0%       97.5%
    a  9.2694878  9.5497650  9.6906850  9.83227750 10.11643500
   bA -1.6852295 -1.3167700 -1.1254650 -0.92889225 -0.53389157
   bM -0.6889247 -0.3151695 -0.1231065  0.07218513  0.45527243
sigma  1.2421182  1.4125950  1.5107700  1.61579000  1.89891925
";

Extract the parameter posterior means: `[β, σ]`,

In [13]:
[posterior_β, posterior_σ]

2-element Array{Any,1}:
  [9.68278, -0.200237, -1.23232]
 1.501953541216366              

end of m4.5d.jl#-
*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*